In [5]:
import livef1 as livef1
season = livef1.get_season(season=2021)
print(season.season_table.head().to_markdown())

|   meeting_key |   season_year | meeting_code   |   meeting_number | meeting_location   | meeting_offname                            | meeting_name       |   meeting_country_key | meeting_country_code   | meeting_country_name   |   meeting_circuit_key | meeting_circuit_shortname   |   session_key | session_type   | session_name   | session_startDate   | session_endDate     | gmtoffset   | path                                                      |
|--------------:|--------------:|:---------------|-----------------:|:-------------------|:-------------------------------------------|:-------------------|----------------------:|:-----------------------|:-----------------------|----------------------:|:----------------------------|--------------:|:---------------|:---------------|:--------------------|:--------------------|:------------|:----------------------------------------------------------|
|          1064 |          2021 | BRN0104        |                1 | Sakhir             | FOR